# Uživatelská příručka
### Spuštění programu

<p> Po spuštění programu budete postupně vyzváni, abyste na samostatný řádek zadali dvě čísla: jedno z nich je rozměr mřížky a druhé pravděpodobnost, s jakou bude při generování mřížky buňka průchozí. Pokud zadáte v jakémkoliv bodě neplatnou hodnotu (například pravděpodobnost větší než 1), bude program ukončen a budete vyzváni k opakovanému spuštění.</p>

<p> Následně budete vyzváni, abyste si zvolili jednu ze tří variant zpracování vstupu zadáním 1, 2 nebo 3. Při jakékoliv volbě se nejprve vytvoří mřížka od zadaných rozměrech, kde každá buňka bude mít šanci na to, že je průchozí podle zadané pravděpodobnosti. Při volbě 1 program zkontroluje, zda by nekonečný proud kapaliny proudící do mřížky zvrchu protekl až do posledního řádku. Pokud ano, pak systém perkoluje. Na výstup se vypíše, zda systém perkoluje či nikoliv a dále se zobrazí ovladatelná animace a statický snímek mřížky po průtoku kapaliny. Animace je ovladatelná skrze tlačítka, která jsou popsaná</p>

<p> Při volbě 2 bude program vyhodnocovat, s jakou pravděpodobností náhodná kapka proteče až do posledního řádku mřížky. Budete vyzváni, abyste zadali počet kapek, které se do systému vpustí. Toto číslo je počet pokusů o protečení kapky, nikoliv počet kapek vpuštěných najednou. Na výstup dostanete hlášení o tom, kolik kapek proteklo až do posledního řádku a procenutální úspěšnost. Rovněž na výstup dostanete statický snímek mřížky, která se vygenerovala.</p>

<p> Při volbě 3 bude program hledat kritickou hodnotu pravděpodobnosti. To znamená, že pro daný rozměr mřížky bude program určovat jaká musí být pravděpodobnost průchodnosti, aby mřížky o daném rozměru perkolovaly s průměrnou úspěšností 50 %. Budete vyzváni, abyste zadali počet iterací tj. počet matic o daném rozměru, které program zhodnotí, aby se dopracoval ke kritické hodnotě. Doporučujeme hodnotu vyšší než 1000. Dále bude program vyžadovat, abyste zadali přesnost, s jakou chcete kritickou hodnotu určit, doporučujeme hodnotu nižší než 0.03, ale nenulovou. Na výstup se vám v průběhu výpočtů budou vypisovat rozsahy, které program prochází. Ty slouží pro orientaci v případech, kdy se program nedobere ke konrétní hodnotě. Pokud se program ke konkrétní hodnotě dobere, dostanete ji na výstup. </p>

### Interpretace výstupů

K vizuálnímu zpracování dat jsou používány teplotní mapy, kde jsou jednotlivé buňky mřížky vykresleny v barvě podle hodnoty, kterou uchovávají. Hodnota 0 odpovídá neprůchozí buňce, tudy nemůže kapalina proudit, hodnota 1 odpovídá průchozí buňce, tudy kapalina proudit může a hodnota 2 znázorňuje buňky, kterými kapalina protekla. Barvy odpovídající jednotlivým hodnotám odečtete ze stupnice po pravé straně teplotních map, které se vám zobrazí na výstupu.

# Porgramátorská dokumentace
<p> Program byl psán za účelem demonstrace perkolace na náhodně vygenerované mřížce. Použity jsou knihovny numpy, plotly a matplotlib. </p>

<p> K vygenerování mřížky byla použita funkce random z knihovny numpy. Mřížku reprezentujeme jako array z knihovny numpy. Jednotlivé stavy buněk jsou reprezentovány hodnotami 0 (neprůchozí), 1 (průchozí), 2 (zaplněná kapalinou). </p>

<p> Funkce flow() vyhodnotí, zda systém perkoluje při průtoku nekonečného proudu kapaliny. Využívá frontu, do které se nejprve uloží všechny průchozí pozice z prvního řádku a následně se vyplní kapalinou. Dokud fronta není prázdná, provádí se následující algoritmus: z fronty se vybere pozice, kde už kapalina je a pokud tato pozice sousedí s nějakými průchozími pozicemi, zaplníme je kapalinou a přidáme je do fronty, aby se prověřily jejich sousedé. Funkce vrací pole (array), kde pozice, kam kapalina potekla, mají hodnotu 2. </p>

<p> Funkce one_drop() provádí pokus, zda náhodná kapka proteče až na poslední řádek, pokud ano, inkrementuje se proměnná uspech, kterou funkce vrací po provedení. Na začátku se do seznamu vacant přidají všechny průchozí pozice z prvního řádku, z těch se následně skrze funkci random.choice vybere jedna, kde bude kapka začínat. Poté následuje while cyklus, dokud kapka není v posledním řádku nebo dokud nemůže dále téct. Pokud je pozice přímo pod kapkou průchozí, kapka proteče, pokud ne, skrze pomocnou funkci check_flow se vybere náhodná průchozí pozice z těch sousedící přímo s aktuální pozicí kapky. Pokud žádná průchozí pozice není, funkce vrátí prázdný seznam a cyklus je ukončen. </p>

<p> Funkce critical_p() pro matice daného rozměru (dim) spočítá kritickou hodnotou pravděpodobnosti, což je hodnota, při které matice o daném rozměru perkoluje s 50% pravděpodobností. Postupně volíme nižší a nižší pravděpodobnosti a u každé vždy spočítáme průměrnou úspěšnost perkolace pro matice velikosti dim pomocnou funkcí perc_succ_rate() pokud je průměrná úspěšnost v námi stanovených mezích (udaných parametrem acc), pak vrátíme aktuální hodnotu pravděpodobnosti. Pokud je průměrná úspěšnost vyšší, další volbou pravděpodobnosti se sníží, pokud je nižší, zmenšíme rozsah, v němž hodnotu hledáme. Celou dobu při každé volbě pravděpodobnosti ukládáme průměrnou úspěšnost a i samotnou pravděpodobnost, abychom mohli vykreslit graf v případě, že to uživatel požaduje. </p>

<p> Funkce heatmap() a animate() využívají knihovnu plotly k vytvoření statického obrázku mřížky a animace, kterou uživatel může ovládat. Obě funkce též využivají objekt Heatmap z knihovny plotly. Snímky animace vznikají z kopií původního pole pod proměnnou matrix obsahující jen hodnoty 0 a 1 a následně se řádky nahrazují řádky z pole fluid_mat(), což je pole vzniklé po provedení funkce flow() na matrix. V každém dalším snímku se nahradí o řádek více. Potom má každý snímek animace o řádek kapaliny navíc. </p>

<p> Funkce run_programme() obsahuje kontroly vstupu. Při chybném zadání vstupu vypíše chybovou hlášku a ukončí se, v opačném případě provede úkony dle volby uživatele. </p>

In [ ]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.setrecursionlimit(10**5)

# Jednotlive stavy jsou v celem kodu reprezentovany nasledovne:  0 - zavreno, 1 - otevreno, 2 - kapalina

def new_matrix(n, p): # Vytvori novou matici n*n s hodnotami 0 a 1, kde je pozice otevrena (1) s pravdepodobnosti p
  
  return np.random.choice([0, 1], size = (n, n), p = [1 - p, p])



def flow(matrix): # Pusti kapalinu a necha ji protect
  queue = []
  d = matrix.shape[0]
  new = np.insert(matrix.copy(), [0, d], -1, axis = 1)

  for i in range(d): # Vsechny otevrene pozice v prvnim radku naplni kapalinou a jejich souradnice da do fronty
    if new[0, i] == 1:
      queue.append([1, i])
      new[0, i] = 2
  
  while len(queue) > 0: # Postupne se z fronty budou vybirat jiz plne pozice a do jejich otevrenych sousedu bude kapalina dale tect
    souradnice = queue.pop(0)
    a, b = souradnice[0], souradnice[1]

    if a == 0:
      if new[a + 1, b] == 1:
        queue.append([a + 1, b])
        new[a + 1, b] = 2
      if new[a, b - 1] == 1:
        queue.append([a, b - 1])
        new[a, b - 1] = 2
      if new[a, b + 1] == 1:
        queue.append([a, b + 1])
        new[a, b + 1] = 2
      if new[a + 1, b + 1] == 1:
        queue.append([a + 1, b + 1])
        new[a + 1, b + 1] = 2
      if new[a + 1, b - 1] == 1:
        queue.append([a + 1, b - 1])
        new[a + 1, b - 1] = 2
    elif a == d - 1:
      if new[a, b + 1] == 1:
        queue.append([a, b + 1])
        new[a, b + 1] = 2
      if new[a, b - 1] == 1:
        queue.append([a, b - 1])
        new[a, b - 1] = 2
    else:
      if new[a, b + 1] == 1:
        queue.append([a, b + 1])
        new[a, b + 1] = 2
      if new[a, b - 1] == 1:
        queue.append([a, b - 1])
        new[a, b - 1] = 2
      if new[a + 1, b] == 1:
        queue.append([a + 1, b])
        new[a + 1, b] = 2
      if new[a + 1, b + 1] == 1:
        queue.append([a + 1, b + 1])
        new[a + 1, b + 1] = 2
      if new[a + 1, b - 1] == 1:
        queue.append([a + 1, b - 1])
        new[a + 1, b - 1] = 2
  
  return np.delete(new, [0, d + 1], axis = 1)



def flow_one_drop(matrix, coords, vaha):
  d = matrix.shape[0]
  x = coords[0]
  y = coords[-1]
  volne = []
  vaha_celkem = 0
  smery = [(0, 1), (0, -1), (1, 1), (1, -1)]

  while True:
    if x == d - 1:
      return vaha
    if matrix[x + 1, y] == 1: # Pokud je pozice pod kapkou volna, tece dal
      x += 1
    else:
      for a, b in smery: # Pokud ne, zkontroluji se ostatni smery a ty, do nichz kapka tect muze, se ulozi
        if matrix[x + a, y + b] == 1:
          volne.append([x + a, y + b])
      
      k = len(volne)
      if k == 0: # Pokud nemuze tect kapka nikam, vrati se nula, protoze kapka neprosla
        return 0
      
      while len(volne) > 0:
        vaha_celkem += flow_one_drop(matrix, volne.pop(), vaha/k) # Jinak se pro kazdou moznou cestu zavola rekurzivne funkce se zmensenou vahou
      
      return vaha_celkem



def one_drop(matrix, num): # Vstupem je matice obsahujici 0 a 1 a num je pocet kapek, ktere do systemu postupne spadnou, vrati pocet kapek, ktere prosly
  d = matrix.shape[0]
  vacant = []
  res = 0

  for i in range(d):
    if matrix[0, i] == 1:
      vacant.append(i)
  
  for i in range(num):
    start = np.random.choice(vacant) # Nahodne zvoli otevrenou pozici v prvnim radku
    new = np.insert(matrix.copy(), [0, d], -1, axis = 1)
    res += flow_one_drop(new, [0, start], 1)
  
  return res



def perc_succ_rate(dim, prob, iter): # Provede iter-krát perkolaci na nahodnych maticich velikosti dim a vrati prumernou uspesnost
  succ = 0

  for i in range(iter):
    mat = new_matrix(dim, prob)
    if percolates_q(mat):
      succ += 1
  
  return succ/iter



def critical_p(dim, iter, acc, draw): # Vrati pribliznou hodnotu pravdepodobnosti, pro kterou matice velikosti dim perkoluje s prumernou uspesnosti 50 %
  spodni = 0.5 - acc
  horni = 0.5 + acc
  zacatek = 0
  krok = 0.5
  konec = 1.1

  x_val = []
  y_val = []

  for i in range(1_000_000):
    for p in np.arange(zacatek, konec, krok): # Vezme se pravdepodobnost z nejakeho rozsahu
      c = perc_succ_rate(dim, 1 - p, iter) # Prumerna uspesnost perkolace pro vybranou pravdepodobnost
      x_val.append(1 - p)
      y_val.append(c)
      if spodni < c < horni: # Pokud je uspesnost dostatecne blizko 0.5, vrati se nam hodnota 
        if draw:
          plt.plot(x_val, y_val, "o", color = "k", markersize = 3)
          plt.plot([1 - p, 1- p], [1, 0], color = "r")
          plt.plot([0, 1], [c, c], color = "r")
          plt.text(1.1 - p, 1.03, "Kritická hodnota: " + str(round(1 - p, 7)))
          plt.text(0.75, c + 0.01, "Průměrná úspěšnost perkolace: " + str(round(c * 100)) + " %")
          plt.xlabel("Pravděpodobnost")
          plt.ylabel("Průměrná úspěšnost perkolace")
          plt.ylim(0, 1)
          plt.show()
        return 1 - p
      elif c < 0.5: # Pokud je uspesnost moc nizko, zuzi se rozsah, v nemz hledame kritickou hodnotu p
        konec = round(p, 5)
        zacatek = round(konec - krok, 5)
        krok /= 3
        konec += krok
        if konec < 1:
          print(f"P je mezi {1 - konec} a {1 - zacatek}")
        break
  print("Program nebyl schopen s vami pozadovanou presnosti urcit konkretni hodnotu p, zkuste zvolit horsi presnost.")
  # Hlaska, pokud ani po opakovanem zuzeni rozsahu nedojdeme k jedine hodnote p.
  # V takovem pripade je mozne cerpat alespon z prubezne se vypisujicich rozsahu, ktere jsou prohledavany.



def percolates_q(matrix): # Vrati True, pokud kapalina protece az na posledni radek
  d = matrix.shape[0]
  vysledna_mat = flow(matrix)

  for i in range(d):
    if vysledna_mat[d - 1, i] == 2:
      return True
  
  return False



def heatmap(matrix, opt): # Vytvori vizualizaci ve forme tepelne mapy, kde je videt stav po prutoku kapaliny
  cmin = np.min(matrix)
  cmax = np.max(matrix)

  if opt == 1:
    nazev = "Perkolace - výsledek"
  else: nazev = "Vygenerovaná matice"
  
  fig = go.Figure()
  mapa = go.Heatmap(z = matrix, colorscale = 'Viridis', showscale = True, zmin = cmin, zmax = cmax)
  
  fig.add_trace(mapa)
  fig.update_layout(title = nazev, xaxis = dict(title = 'Sloupec'), yaxis = dict(title = 'Řádek', autorange = 'reversed'), width = 500, height = 500)
  fig.show()



def animate(matrix): # Vytvori ovladatelnou animaci, kde je videt, jak kapalina postupne proteka zeshora dolu
  # matrix je vygenerovana matice, tedy obsahuje jen 0 a 1

  frames = []
  d = matrix.shape[0]
  fluid_mat = flow(matrix) # Ulozime si podobu matice po prutoku kapaliny
  first_frame = matrix.copy()
  first_frame[0, :] = fluid_mat[0, :] # V prvnim snimku bude kapalina jen v hornim radku

  cmin = np.min(first_frame)
  cmax = np.max(first_frame)

  for i in range(1, d): # Novy snimek se vytvori tak, ze se ulozi kopie matrix a prvních i radku se nahradi radky fluid_mat, aby byla kapalina vzdy jen o radek nize
    next_frame = matrix.copy()
    next_frame[:i + 1, :] = fluid_mat[:i + 1, :]
    frames.append(go.Frame(data = [go.Heatmap(z = next_frame)]))
  
  fig = make_subplots(rows = 1, cols = 1)
  mapa = go.Heatmap(z = first_frame, colorscale = 'Viridis', showscale = True, zmin = cmin, zmax = cmax)
  fig.add_trace(mapa)

  fig.update_layout(title = "Perkolace - animace", xaxis = dict(title = "Sloupec"), yaxis = dict(title = "Řádek", autorange = 'reversed'), width = 500, height = 500,
  updatemenus = [dict(type = 'buttons', showactive = False, buttons = [dict(label = 'Spustit', method = 'animate',
    args = [None, dict(frame = dict(duration = 80, redraw = True), fromcurrent = True, mode = 'immediate')]),
    dict(label = 'Zastavit', method = 'animate', args = [[None], dict(frame = dict(duration = 0, redraw = False), mode = 'immediate')])])])
  
  fig.frames = frames
  fig.show()



def run_programme(): # Funkce, ktera se spusti zaroven se spustenim programu a zastavi cely program v momente, kdy dojde k chybe pri zadavani
  dim = int(input("Zadejte rozmer mrizky (pritozene cislo):"))
  if dim <= 0:
    print("Chybny vstup, zkuste to znovu.")
    return
  
  pst = float(input("Zadejte pravdepodobnost pruchodnosti pozice (cislo mezi 0 a 1):"))
  if pst < 0 or pst > 1:
    print("Chybny vstup, zkuste to znovu.")
    return
  
  choice = int(input("Pokud chcete vysetrit perkolaci, zadejte 1, pokud chcete spocitat pruchodnost mrizky, zadejte 2, pokud chcete nalezt kritickou hodnotu p, zadejte 3."))
  if choice != 1 and choice != 2 and choice != 3:
    print("Chybny vstup, zkuste to znovu.")
    return
  
  matice = new_matrix(dim, pst)

  if choice == 1:
    if percolates_q(matice):
      print("System perkoluje.")
    else:
      print("System neperkoluje.")

    animate(matice)
    heatmap(flow(matice), choice)
    return
  elif choice == 2:
    pocet = int(input("Zadejte pocet testu pruchodu (prirozene cislo)"))
    if pocet <= 0:
      print("Chybny vstup, zkuste to znovu.")
      return
    odpor = one_drop(matice, pocet)
    print(f"Hydraulický odpor mřížky: {1 - (odpor/pocet)}")
    heatmap(matice, choice)
    return
  else:
    iterace = int(input("Zadejte pocet iteraci (prirozene cislo, doporucena hodnota 1000 a vice)"))
    if iterace <= 0:
      print("Chybny vstup, zkuste to znovu.")
      return
    presnost = float(input("Zadejte prenost vypoctu (doporucena hodnota mezi 0 a 0.01)"))
    if presnost <= 0 or presnost >= 0.5:
      print("Chybny vstup, zkuste to znovu.")
      return
    b = str(input("Chcete vykreslit graf? A = ANO, N = NE"))
    if b == 'a' or b == 'A':
      graf = True
    elif b == 'n' or b == 'N':
      graf = False
    else:
      print("Chybny vstup, zkuste to znovu.")
      return
    p = critical_p(dim, iterace, presnost, graf)
    if not graf:
      print(f"Kriticka hodnota pravdepodobnosti pro mrizky rozmeru {dim} je {p}")
    return

# run_programme()

# Funkce s jednou castici neni zatim soucasti funkce run_programme, protoze nefunguje, muzete ji spustit pres zakomentovanou cast nize
# dim = 50
# p = 0.45
# num = 100
# print(one_drop(new_matrix(dim, p), num))